In [2]:
def add_sub_list(list):
    l = []
    [l.append(x) for x in list]
    return l

In [16]:
import os
import json
import subprocess
import numpy as np
import pandas as pd
from skimage.measure import find_contours
import shutil

class CocoDatasetHandler:

    def __init__(self, jsonpath, imgpath):
        #  CocoDatasetHandler('./test_0/_annotations.coco.json', 'test_0/')
        with open(jsonpath, 'r') as jsonfile:
            ann = json.load(jsonfile)

        images = pd.DataFrame.from_dict(ann['images']).set_index('id')
        annotations = pd.DataFrame.from_dict(ann['annotations']).set_index('id')
        categories = pd.DataFrame.from_dict(ann['categories']).set_index('id')

        # print(images)
        # print(annotations)
        # print(categories)
        annotations = annotations.merge(categories, left_on='category_id', right_index=True)
        # annotations = annotations.groupby('image_id').agg({'bbox': lambda x: add_sub_list(x), 'name': lambda x: add_sub_list(x)})

        annotations = annotations.merge(images, left_on='image_id', right_index=True)
        annotations = annotations.assign(
            shapes=annotations.apply(self.coco2shape, axis=1))


        self.annotations = annotations
        self.labelme = {}
        self.roboflow = []
        self.imgpath = imgpath
        self.images = pd.DataFrame.from_dict(ann['images']).set_index('file_name')

    def coco2shape(self, row):
        
        if row.iscrowd == 1:
            shapes = self.rle2shape(row)
        elif row.iscrowd == 0:
            shapes = self.polygon2shape(row)
        return shapes

    def rle2shape(self, row):
        print("rle",row)
        rle, shape = row['segmentation']['counts'], row['segmentation']['size']
        mask = self._rle_decode(rle, shape)
        padded_mask = np.zeros(
            (mask.shape[0]+2, mask.shape[1]+2),
            dtype=np.uint8,
        )
        padded_mask[1:-1, 1:-1] = mask
        points = find_contours(mask, 0.5)
        shapes = [
            [[int(point[1]), int(point[0])] for point in polygon]
            for polygon in points
        ]
        return shapes

    def _rle_decode(self, rle, shape):
        mask = np.zeros([shape[0] * shape[1]], np.bool)
        for idx, r in enumerate(rle):
            if idx < 1:
                s = 0
            else:
                s = sum(rle[:idx])
            e = s + r
            if e == s:
                continue
            assert 0 <= s < mask.shape[0]
            assert 1 <= e <= mask.shape[0], "shape: {}  s {}  e {} r {}".format(shape, s, e, r)
            if idx % 2 == 1:
                mask[s:e] = 1
        # Reshape and transpose
        mask = mask.reshape([shape[1], shape[0]]).T
        return mask

    def polygon2shape(self, row):
        # print("poligon",row.image_id)
        # shapes: (n_polygons, n_points, 2)

        # coco: 
        # For object detection annotations, the format is "bbox" : [x,y,width,height]
        # Where:
        # x, y: the upper-left coordinates of the bounding box
        # width, height: the dimensions of your bounding box

        # {
        # "label": "isolated_expression",
        # "points": [
        #     [
        #     439.61352657004835,
        #     939.1304347826087
        #     ],
        #     [
        #     960.8695652173914,
        #     970.048309178744
        #     ]
        # ],
        # "group_id": null,
        # "shape_type": "rectangle",
        # "flags": {}
        # },
        # names = row.name
        # bbox = row.bbox 
        # l = len(row.name)
        # shapes = [
        #     {
        #         "label": names[i],
        #         "points": [[bbox[0], bbox[1]], [bbox[0] + bbox[2], bbox[1] + bbox[3]]],
        #         "group_id": None,
        #         "shape_type" : "rectangle",
        #         "flags": {}
        #     }
        #     for i in range(l)
        # ]
        shapes = [
            [
                [round(row.bbox[0],2), round(row.bbox[1],2)],
                [round(row.bbox[0] + row.bbox[2], 2), round(row.bbox[1] + row.bbox[3], 2)]
            ]
        ]
        # print(shapes)
        return shapes

    def coco2labelme(self):
        fillColor = [255, 0, 0, 128]
        lineColor = [0, 255, 0, 128]

        groups = self.annotations.groupby('file_name')
        
        for file_idx, (filename, df) in enumerate(groups):
            
            name = filename.split('_',1)[0]+".jpg"
            record = {
                'imageData': None,
                'fillColor': fillColor,
                'lineColor': lineColor,
                'imagePath': name,
                'imageHeight': int(self.images.loc[filename].height),
                'imageWidth': int(self.images.loc[filename].width),
            }
            record['shapes'] = []

            instance = {
                'line_color': None,
                'fill_color': None,
                'shape_type': "rectangle",
            }


            for inst_idx, (_, row) in enumerate(df.iterrows()):
                # print(row)
                for polygon in row.shapes:
                    copy_instance = instance.copy()
                    copy_instance.update({
                        'label': row['name'],
                        'group_id': inst_idx,
                        'points': polygon
                    })
                    record['shapes'].append(copy_instance)
            if filename not in self.roboflow:
                self.roboflow.append(filename)
            if name not in self.labelme.keys():
                self.labelme[name] = record

    def save_labelme(self, file_names, dirpath, save_json_only=False):
        # print(os.path.dirname(os.path.abspath(__file__)))
        
        # uncomment if dir not exists
        
        # if not os.path.exists(dirpath):
        #     os.makedirs(dirpath)
        # else:
        #     raise ValueError(f"{dirpath} has existed")


        for file in file_names:
            filename = os.path.basename(os.path.splitext(file)[0])
            print(filename)
            name = filename.split('_')[0]
            newfile = name+".jpg"

            with open(os.path.join(dirpath, name+'.json'), 'w') as jsonfile:
                json.dump(self.labelme[newfile], jsonfile, ensure_ascii=True, indent=2)

            if not save_json_only:
                print(os.path.join(self.imgpath, file))
                
                # shutil.copy2('/src/dir/file.ext', '/dst/dir/newname.ext') # complete target filename given
                # shutil.copy2('/src/file.ext', '/dst/dir') # target filename is /dst/dir/file.ext
                
                shutil.copy2(os.path.join(self.imgpath, file), dirpath) # target filename is /dst/dir/file.ext

                # subprocess.call(['cp', os.path.join(self.imgpath, file), dirpath])
                # comment if file name is clean (not from roboflow)
                os.rename(os.path.join(dirpath, file), os.path.join(dirpath, newfile))

In [21]:


ds = CocoDatasetHandler('./valid_0/_annotations.coco.json', 'valid_0/')
# print(ds.annotations)


In [22]:
ds.coco2labelme()


In [23]:
ds.save_labelme(ds.roboflow, 'labelme')

102_png.rf.3bc41792142408872102e8dcb97e5f38
valid_0/102_png.rf.3bc41792142408872102e8dcb97e5f38.jpg
107_png.rf.6e6d4ff8094063afb49996b5ef7da94d
valid_0/107_png.rf.6e6d4ff8094063afb49996b5ef7da94d.jpg
117_png.rf.f473937a6c78ffcee7769671a1fbcdc5
valid_0/117_png.rf.f473937a6c78ffcee7769671a1fbcdc5.jpg
119_png.rf.9bf449b63b20673056efa0151b48d591
valid_0/119_png.rf.9bf449b63b20673056efa0151b48d591.jpg
122_png.rf.eb0e20c63c6983cf96d97b7a287a5f05
valid_0/122_png.rf.eb0e20c63c6983cf96d97b7a287a5f05.jpg
126_png.rf.e219f4b588490e3e963b6e12171e64d2
valid_0/126_png.rf.e219f4b588490e3e963b6e12171e64d2.jpg
12_png.rf.f43e57f937343801b6bcdeba48d21fcf
valid_0/12_png.rf.f43e57f937343801b6bcdeba48d21fcf.jpg
139_png.rf.36057bafc0d15d69d702737642542289
valid_0/139_png.rf.36057bafc0d15d69d702737642542289.jpg
140_png.rf.48563489f1e2aa87ab9ac8ad6c519ab2
valid_0/140_png.rf.48563489f1e2aa87ab9ac8ad6c519ab2.jpg
142_png.rf.90ecb963ea4dc27b18963486ac7e2f21
valid_0/142_png.rf.90ecb963ea4dc27b18963486ac7e2f21.jpg
17